In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
import anndata as ad

In [ ]:
binary_matrix = pd.read_csv('../MOFA/MOFA_15_Factors_6082HVG/binary_matrix_factors.csv', index_col='Unnamed: 0')
adata = sc.read_h5ad('../MOFA/adata_combined_even_more_hvg.h5ad')

In [ ]:
genes_mofa_factors = binary_matrix.T.columns.tolist()

In [ ]:
pd.DataFrame(genes_mofa_factors).to_csv('MOFA_genes.csv')

In [ ]:
len(genes_mofa_factors)

# DE genes across Technologies

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='Condition', layer='log_norm')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby='Condition', values_to_plot='logfoldchanges', n_genes=100)

In [ ]:
df = pd.DataFrame(adata.uns['rank_genes_groups']['names'])

In [ ]:
df_sc = sc.get.rank_genes_groups_df(adata, group='scRNA-seq')
df_sn = sc.get.rank_genes_groups_df(adata, group='snRNA-seq')

In [ ]:
de_sc = df_sc[df_sc.logfoldchanges>5].names.tolist()
de_sn = df_sn[df_sn.logfoldchanges>5].names.tolist()

In [ ]:
sc_remove = ([i for i in de_sc if i in genes_mofa_factors])
sn_remove = ([i for i in de_sn if i in genes_mofa_factors])

In [ ]:
genes_to_remove_techincal_variability= sn_remove + sc_remove

In [ ]:
len(genes_to_remove_techincal_variability)

In [ ]:
sc.pl.dotplot(adata, groupby='Level_1', var_names=genes_to_remove_techincal_variability,standard_scale='var', layer='log_norm' )

# Markers from Broad

In [ ]:
broad_markers = pd.read_csv('broad_markers.csv', index_col='Unnamed: 0')

In [ ]:
broad_markers

In [ ]:
broad_markers_list = list(set(broad_markers.values.flatten().astype(str)[broad_markers.values.flatten().astype(str) != 'nan']))

In [ ]:
pd.DataFrame(broad_markers_list).to_csv('broad_markers_list.csv')

In [ ]:
([i for i in broad_markers_list if i not in adata.var_names])

# DE Markers across Cell Types

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='Level_1', layer='log_norm')

In [ ]:
df = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(30)
print(len(list(set(df.values.flatten()))))

In [ ]:
# Filter out genes that are already present in genes_mofa_factors 
genes_to_check = ([i for i in (list(set(df.values.flatten()))) if i not in genes_mofa_factors])
print(len(genes_to_check))

In [ ]:
df.head()

In [ ]:
gene_to_group = {}
for gene in genes_to_check:
    test= df.where(df==gene).dropna(how='all').dropna(axis=1, how='all')
    cols = test.columns.tolist()
    gene_to_group[gene] = cols

In [ ]:
df['Adipocyte'][df['Adipocyte'].isin(genes_to_check)].tolist()

In [ ]:
gene_to_group_T = {}
for cols in df.columns:
    temp = df[cols]
    temp_genes = temp[temp.isin(genes_to_check)].tolist()
    gene_to_group_T[cols] = temp_genes

In [ ]:
df = pd.DataFrame.from_dict(gene_to_group_T, orient='index')

In [ ]:
de_genes = list(set([val for val in df.values.flatten() if val is not None]))
print(type(de_genes))

In [ ]:
df_genes = pd.DataFrame(de_genes, columns=['Gene'])

In [ ]:
df_genes.reset_index(inplace=True)

In [ ]:
print(df_genes.dtypes)

In [ ]:
df_genes.to_csv('de_genes_to_be_added.csv')

In [ ]:
from collections import Counter
Counter(pd.DataFrame.from_dict(gene_to_group, orient='index').values.flatten())

# Check Genes

In [ ]:
all_genes = (genes_mofa_factors + broad_markers_list + de_genes)

In [ ]:
len([i for i in list(set(all_genes)) if i not in adata.var_names])

In [ ]:
[i for i in all_genes if i not in adata.var_names]

# Reload

In [ ]:
mofa_genes_df = pd.read_csv('MOFA_genes.csv')
broad_markers = pd.read_csv('broad_markers.csv', index_col='Unnamed: 0')
broad_markers_list = list(set(broad_markers.values.flatten().astype(str)[broad_markers.values.flatten().astype(str) != 'nan']))
de_genes_df = pd.read_pickle('de_genes_to_be_added.csv')
mofa_genes = mofa_genes_df.values.flatten().tolist()
de_genes = de_genes_df.values.flatten().tolist()

In [ ]:
all_genes = (genes_mofa_factors + broad_markers_list + de_genes)

# Xenium Panel

In [ ]:
xenium = pd.read_csv('pdac_xenium_panel.csv')

In [ ]:
xenium_genes = list(set(xenium.Gene.tolist()))

In [ ]:
len(xenium_genes)

In [ ]:
# check how many genes not in all_genes
len([i for i in xenium_genes if i not in all_genes])

In [ ]:
valid_genes = [i for i in xenium_genes if i in adata.var_names]

In [ ]:
genes_to_plot = [i for i in valid_genes if i not in all_genes]

In [ ]:
# Number of xenium genes present in 6082 HVG and in the selected all_genes list
len(genes_to_plot)

In [ ]:
sc.pl.dotplot(adata, groupby='Level_1', var_names=genes_to_plot, layer='log_norm', standard_scale='var')

# Create final list

In [ ]:
all_genes = list(set(mofa_genes + broad_markers_list + de_genes + xenium_genes))

In [ ]:
len(all_genes)

In [ ]:
all_genes = list(set(mofa_genes + broad_markers_list + de_genes + xenium_genes))

In [ ]:
all_genes = [i for i in all_genes if i in adata.var_names]